# Queens Problem

## Import libraries

In [172]:
import numpy as np
import random

## Define Agent, population and display

In [173]:
def init_population(initial_population_size, board_size):
    population = []
    for i in range(initial_population_size):
      agent = [x for x in range(1, BOARD_SIZE+1)]
      random.shuffle(agent)
      population.append(agent)
    return population

def print_board(board_size, agent):
    for y in range(1,1+board_size):
      row = []
      for x in range(board_size):
        if agent.index(y) == x:
          row.append(1)
        else:
          row.append(0)
      print(row)

## Define crossover

In [174]:
def crossover(parent1, parent2):
    child1 = parent1[:len(parent1)//2]
    child2 = parent2[:len(parent2)//2]
    for i in parent2:
      if i not in child1:
        child1.append(i)
    for i in parent1:
      if i not in child2:
        child2.append(i)
    return child1, child2

## Define mutation

In [175]:
def mutation(agent):
  variety = random.random()
  if variety < 0.33:
    position1 = random.randint(0, len(agent) - 1)
    position2 = random.randint(0, len(agent) - 1)
    while position1 == position2:
      position2 = random.randint(0, len(agent) - 1)
    agent[position1], agent[position2] = agent[position2], agent[position1]
  elif variety < 0.66:
    start = random.randint(0, len(agent) - 1)
    end = random.randint(start, len(agent))
    agent[start:end] = reversed(agent[start:end])
  else:
    random.shuffle(agent)
  return agent

def mutation_with_chance(agent, mutation_chance):
    if random.random() < MUTATION_CHANCE:
      agent = mutation(agent)
    return agent

## Define crossover of the entire population with mutation

In [176]:
def replicate(population, mutation_chance):
  random.shuffle(population)
  parents = list(range(1, len(population),2))
  random_parent = random.choice(parents)
  parents.remove(random_parent)
  for i in range(0,len(population),2):
    parent1 = population[i]
    parent2 = population[random_parent]
    child1,child2 = crossover(parent1, parent2)
    mutation_with_chance(child1, mutation_chance)
    mutation_with_chance(child2, mutation_chance)
    population.append(child1)
    population.append(child2)
  return population

## Define fitness function

In [177]:
def fitness_agent(agent):
    collisions = 0
    for y in agent:
      for x in range(len(agent)):
        if abs(y-agent[x]) == abs(x-agent.index(y)):
          collisions += 1
    return collisions-len(agent)

def fitness_population(population):
    fitness_list = []
    for agent in population:
      fitness_list.append(fitness_agent(agent))

    return fitness_list

## Population Selection (by score)

In [178]:
def sort_population(population, fitness_list):
    fitness_list = sorted(zip(population, fitness_list),key = lambda x:x[1])
    return fitness_list

def select_population(population, fitness_list, best_pop, mid_pop, least_pop, random_pop):
    global INITIAL_POPULATION
    sorted_population = sort_population(population, fitness_list)
    new_gen, fitness = zip(*sorted_population)
    new_gen = list(new_gen)
    if fitness[0] != 0:
      best_gen = new_gen[:int(INITIAL_POPULATION*best_pop)]
      mid_gen = random.sample(new_gen[int(INITIAL_POPULATION*best_pop):-int(INITIAL_POPULATION*least_pop)], int(mid_pop*INITIAL_POPULATION))
      least_gen = new_gen[-int(INITIAL_POPULATION*least_pop):]
      random_gen = init_population(int(INITIAL_POPULATION*random_pop), BOARD_SIZE)
      random.shuffle(random_gen)
      selected_pop = best_gen + mid_gen + least_gen + random_gen
      return list(selected_pop), fitness[0]
    else:
      return new_gen, fitness[0]

## Define main

In [179]:
def main(population_size, board_size, mutation_chance, best_pop, mid_pop, least_pop, random_pop):
    population = init_population(INITIAL_POPULATION, BOARD_SIZE)
    best_fitness = 1
    last_fitness = 0
    count_fitness = 0
    iteration = 0

    while best_fitness > 0:
      replicate(population, MUTATION_CHANCE)
      fitness_list = fitness_population(population)
      population, fitness = select_population(population, fitness_list, best_pop, mid_pop, least_pop, random_pop)
      best_fitness = fitness

      if iteration % 100 == 0:
        print(f'Iteration {iteration} -> Best fitness {best_fitness} -> Population: {len(population)}')
        if mutation_chance < 0.4 and count_fitness == 4:
          mutation_chance += 0.1
          print(f"Mutation increased to {int(mutation_chance*100)}%")
          count_fitness = 0
        elif mutation_chance == 0.4 and count_fitness == 8 and best_fitness == 2:
            print("Mutation chance is at maximum (40%) reseting population and mutation...")
            population = init_population(INITIAL_POPULATION, BOARD_SIZE)
            mutation_chance = 0.1
            count_fitness = 0
        elif last_fitness == best_fitness:
            count_fitness += 1
        else:
            last_fitness = best_fitness

      iteration += 1
    print(f"A valid gene {population[0]} with fitness {fitness} has been found in iteration {iteration}")
    print_board(board_size, population[0])

## Define global variables and test code

In [ ]:
INITIAL_POPULATION = 100
BOARD_SIZE = 20
MUTATION_CHANCE = 0.1
best_pop = 0.5
mid_pop = 0.15
least_pop = 0.10
random_pop = 0.25

main(INITIAL_POPULATION, BOARD_SIZE, MUTATION_CHANCE, best_pop, mid_pop, least_pop, random_pop)

Iteration 0 -> Best fitness 6 -> Population: 100
Iteration 100 -> Best fitness 4 -> Population: 100
Iteration 200 -> Best fitness 2 -> Population: 100
Iteration 300 -> Best fitness 2 -> Population: 100
Iteration 400 -> Best fitness 2 -> Population: 100
Iteration 500 -> Best fitness 2 -> Population: 100
Iteration 600 -> Best fitness 2 -> Population: 100
Iteration 700 -> Best fitness 2 -> Population: 100
Mutation increased to 20%
Iteration 800 -> Best fitness 2 -> Population: 100
Iteration 900 -> Best fitness 2 -> Population: 100
Iteration 1000 -> Best fitness 2 -> Population: 100
Iteration 1100 -> Best fitness 2 -> Population: 100
Iteration 1200 -> Best fitness 2 -> Population: 100
Mutation increased to 30%
Iteration 1300 -> Best fitness 2 -> Population: 100
Iteration 1400 -> Best fitness 2 -> Population: 100
Iteration 1500 -> Best fitness 2 -> Population: 100
Iteration 1600 -> Best fitness 2 -> Population: 100
Iteration 1700 -> Best fitness 2 -> Population: 100
Mutation increased to 40